## Travail sur les modèles décisionnels

### Problématique :
-- SAISIR ICI LA PROBLÉMATIQUE --

### Prérequis :
* Le script suppose d'avoir télécharger les données issues de l'entrepôt sur la plateforme datagrosyt (https://agrosyst.fr/datagrosyst/)

* Le script est rédigé en **Python** et suppose une bonne maîtrise de **pandas** 


#### Import des librairies

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np

#### Import des données

In [11]:
# ------------------ #
# IMPORT DES DONNÉES #
# ------------------ #
ENTREPOT_PATH = '~/Bureau/utils/data/'
df = {}

def import_df(df_name, path_data, sep, index_col=None):
    df[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False).replace({'\r\n': '\n'}, regex=True)

def import_dfs(df_names, path_data, sep = ',', index_col=None, verbose=False):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(" - ", df_name)
        import_df(df_name, path_data, sep, index_col=index_col)

tables_with_id = [
    'modele_decisionnel', 
    'modele_decisionnel_maitrise', 
    'modele_decisionnel_strategie', 
]

tables_without_id = [
    'modele_decisionnel_strategie_culture'
]

# import des données de l'entrepôt avec la colonne 'id' en index 
import_dfs(tables_with_id, ENTREPOT_PATH, sep = ',', index_col='id', verbose=False)

# import des données du magasin
import_dfs(tables_without_id, ENTREPOT_PATH, sep = ',', verbose=False)

100%|██████████| 1/1 [00:00<00:00,  1.54it/s]


#### Traitement des données